# <center>**Taller 06: Bases de Datos Vectoriales**</center>

### **Integrantes:**
- Calahorrano David
- Códorva Carlos
- Zambrano Ándres

### **Objetivo:**
En este ejercicio implementarás un sistema de recuperación que utilice representaciones vectoriales de texto y compararás este enfoque con los modelos TF-IDF y BM25.

### **Requisitos:**
1. Librerías Python necesarias: sentence-transformers, FAISS, ChromaDB.
2. Instalar Elasticsearch en un contenedor utilizando Docker.
3. Dataset Wikipedia Movie Plots, disponible en Kaggle, y utilizado en el taller 05.

---

### **Conceptos Clave**
#### **TF-IDF**
- **Definición:** TF-IDF (Term Frequency - Inverse Document Frequency) es una métrica que mide la relevancia de un término dentro de un documento respecto a un corpus. Combina dos factores:
- **Frecuencia del término (TF):** Qué tan frecuente es una palabra en un documento.
- **Frecuencia inversa en el corpus (IDF):** Penaliza palabras comunes en el corpus.

#### **BM25**
- **Definición:** Una extensión de TF-IDF que ajusta la relevancia basada en:
  - La longitud del documento (normalización).
  - La saturación de la frecuencia del término (evita que palabras muy repetidas dominen el puntaje).
- **Ventajas:** Proporciona una búsqueda de texto más precisa y es el método predeterminado en Elasticsearch.

#### **Embeddings**
- **Definición:** Representaciones vectoriales de texto que capturan relaciones semánticas entre palabras o frases. Los embeddings transforman palabras o documentos en vectores en un espacio multidimensional donde textos similares están cerca entre sí.
- **Herramienta:** Librería sentence-transformers, que ofrece modelos preentrenados como all-MiniLM-L6-v2 para generar embeddings de alta calidad.

#### **Bases de Datos Vectoriales (FAISS y ChromaDB)**
- **FAISS:** Biblioteca especializada en búsquedas rápidas en grandes colecciones de vectores. Diseñada para escalabilidad y velocidad.
- **ChromaDB:** Base de datos orientada a embeddings que ofrece funcionalidades avanzadas para almacenar y consultar vectores de manera eficiente.

#### **Comparación de Estrategias de Recuperación**
- **TF-IDF vs. BM25 vs. Embeddings:**
  - **TF-IDF:** Mide la relevancia de palabras clave basándose en su frecuencia relativa.
  - **BM25:** Mejora TF-IDF ajustando por la longitud del documento y saturación de términos.
  - **Embeddings:** Capturan relaciones semánticas, permitiendo recuperar documentos relevantes, aunque las palabras exactas no coincidan.

#### **Preparación del Dataset: Wikipedia Movie Plots**
1. Usa el dataset del Taller 05.

In [1]:
import pandas as pd
# Cargar el dataset utilizado en el taller 05
file_path = r'C:\Users\USER\.cache\kagglehub\datasets\jrobischon\wikipedia-movie-plots\versions\1\wiki_movie_plots_deduped.csv'
df = pd.read_csv(file_path)
data = df[['Title', 'Plot']].head(36000)

In [2]:
data.head(1000)

,Title,Plot
0,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr..."
1,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov..."
2,The Martyred Presidents,"The film, just over a minute long, is composed..."
3,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...
4,Jack and the Beanstalk,The earliest known adaptation of the classic f...
...,...,...
995,Playing Around,Alice White plays the part of a working class ...
996,Raffles,Gentleman jewel thief A.J. Raffles (Ronald Col...
997,Reaching for the Moon,"Wall Street wizard, Larry Day, new to the ways..."
998,Recaptured Love,"In this drama, a 50-year-old married man (play..."


In [3]:
import re

def normalize_text(text):
    text = text.lower()  # Minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuaciones
    text = re.sub(r'\d+', '', text)     # Eliminar números
    text = re.sub(r'\s+', ' ', text).strip()  # Espacios adicionales
    
    # Eliminar palabras repetidas
    palabras = text.split()
    palabras_unicas = []
    for palabra in palabras:
        if palabra not in palabras_unicas:
            palabras_unicas.append(palabra)
    return ' '.join(palabras_unicas)

# Normalizar tramas
data['Normalized Plot'] = data['Plot'].apply(normalize_text)

---

### **Instrucciones**
#### **Parte 1: Recuperación con TF-IDF**
1. Cargar los datos en Python
2. Configurar TF-IDF:
   - Usa la librería scikit-learn para calcular los puntajes TF-IDF de los plots.
3. Realizar consultas:
   - Escribe una función que calcule la similitud entre una consulta y los documentos usando la matriz TF-IDF.
4. Evaluar los resultados:
   - Registra los documentos recuperados y analiza su relevancia.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(data['Normalized Plot'])

# Consulta con TF-IDF
query = "time travel future"
query_vec = vectorizer.transform([normalize_text(query)])

# Calcular similitud de coseno
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(query_vec, X_tfidf).flatten()

# Obtener los resultados más relevantes
results_tfidf = data.iloc[cos_sim.argsort()[-10:][::-1]]['Title']
print("Resultados con TF-IDF:")
print(results_tfidf)

Resultados con TF-IDF:
33572     Yona Yona Penguin
7771                Rampage
649       The Love of Sunya
17507             Crosstalk
3799              Girl Rush
10773          Mr. Nice Guy
18620    The Night Has Eyes
29474         Kurathi Magan
29310                Bommai
22010               XChange
Name: Title, dtype: object


#### **Parte 2: Recuperación con BM25**
1. Configurar Elasticsearch:
   - Reutiliza el índice creado en el Ejercicio 1 para realizar consultas basadas en BM25.
2. Realizar consultas:
   - Usa palabras clave como "dinosaurs" o "cyborg" para encontrar documentos relevantes.
3. Evaluar los resultados:
   - Compara los documentos recuperados con los obtenidos usando TF-IDF.

In [5]:
from elasticsearch import Elasticsearch

# Crear la conexión
es = Elasticsearch(['http://localhost:9200'])  # Se incluye el esquema http
index_name = 'movies'

# Verificar la conexión
if es.ping():
    print("Conectado a Elasticsearch")
else:
    print("Error al conectar con Elasticsearch")

Conectado a Elasticsearch


In [6]:
# Crear índice
def crear_indice_elasticsearch():
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name, body={
            "mappings": {
                "properties": {
                    "title": {"type": "text"},
                    "plot": {"type": "text"}
                }
            }
        })

In [7]:
from elasticsearch.helpers import bulk

# Indexar documentos
def indexar_documentos(data):
    actions = []
    for _, row in data.iterrows():
        actions.append({
            "_index": index_name,
            "_source": {
                "title": row['Title'],
                "plot": row['Normalized Plot']
            }
        })
    bulk(es, actions)

crear_indice_elasticsearch()
indexar_documentos(data)

In [8]:
def buscar_bm25_elasticsearch(consulta):
    query = {
        "query": {
            "match": {
                "plot": consulta
            }
        }
    }
    resultados = es.search(index=index_name, body=query, size=10)
    for hit in resultados['hits']['hits']:
        print(hit['_source']['title'])

In [9]:
# Realizar consultas
consulta_bm25 = "dinosaurs"
print("Resultados con BM25 para la consulta 'dinosaurs':")
buscar_bm25_elasticsearch(consulta_bm25)

Resultados con BM25 para la consulta 'dinosaurs':


C:\Users\USER\AppData\Local\Temp\ipykernel_20804\268924553.py:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resultados = es.search(index=index_name, body=query, size=10)


Theodore Rex
Theodore Rex
Theodore Rex
Two Lost Worlds
Two Lost Worlds
Two Lost Worlds
The Lost World
Magic Tree House
The Lost World
Magic Tree House


In [10]:
consulta_bm25_2 = "cyborg"
print("Resultados con BM25 para la consulta 'cyborg':")
buscar_bm25_elasticsearch(consulta_bm25_2)

Resultados con BM25 para la consulta 'cyborg':
Cyborg 3: The Recycler
Future X-Cops
Cyborg 3: The Recycler
Future X-Cops
Cyborg 3: The Recycler
Future X-Cops
Sun Vulcan Movie
964 Pinocchio
Sun Vulcan Movie
964 Pinocchio


C:\Users\USER\AppData\Local\Temp\ipykernel_20804\268924553.py:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resultados = es.search(index=index_name, body=query, size=10)


#### **Parte 3: Recuperación con FAISS**
1. Configurar FAISS:
   - Crea un índice en FAISS y agrega los embeddings generados previamente.
2. Realizar consultas:
   - Convierte una consulta en texto (e.g., "A park with cloned dinosaurs") en un vector usando el mismo modelo de embeddings.
   - Busca los vectores más cercanos en FAISS.
3. Evaluar los resultados:
   - Compara los documentos recuperados por FAISS con los obtenidos usando TF-IDF y BM25.

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Configurar el modelo
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings para cada trama normalizada
data['Embeddings'] = data['Normalized Plot'].apply(lambda x: model.encode(x))

# Crear un índice en FAISS
dimension = len(data['Embeddings'][0])  # Dimensión de los embeddings
index = faiss.IndexFlatL2(dimension)   # Índice de FAISS usando L2 (distancia euclidiana)

# Convertir los embeddings en una matriz numpy y añadirlos al índice
embeddings_matrix = np.array(data['Embeddings'].tolist())
index.add(embeddings_matrix)
print("Embeddings indexados en FAISS.")

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from sentence_transformers import SentenceTransformer

# Cargar modelo de prueba
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Modelo cargado correctamente.")

# Probar generación de embeddings
query = "A park with cloned dinosaurs"
query_embedding = model.encode(query)
print("Primeros 5 valores del embedding:", query_embedding[:5])

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Configurar FAISS y generar embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
data['Embeddings'] = data['Normalized Plot'].apply(lambda x: model.encode(x))

# Crear el índice de FAISS
dimension = len(data['Embeddings'][0])
index = faiss.IndexFlatL2(dimension)

# Convertir embeddings a formato numpy y añadirlos al índice
embeddings_matrix = data['Embeddings'].tolist()
index.add(np.array(embeddings_matrix))

# Realizar consultas
query_text = "A park with cloned dinosaurs"
query_embedding = model.encode(query_text)

# Buscar los vectores más cercanos
k = 10  # Número de resultados
distances, indices = index.search(np.array([query_embedding]), k)

#### **Parte 4: Recuperación con ChromaDB**
1. Configurar ChromaDB:
   - Inicia una base de datos de ChromaDB y define el esquema con los campos Title, Plot y Embedding.
2. Insertar documentos y embeddings:
   - Agrega los documentos y sus embeddings generados previamente a ChromaDB, junto con los metadatos correspondientes (e.g., título y trama).
3. Realizar consultas:
   - Convierte una consulta de texto en un embedding.
   - Busca los vectores más cercanos en ChromaDB y recupera los documentos relacionados.
4. Evaluar los resultados:
   - Compara los documentos recuperados por ChromaDB con los de FAISS, TF-IDF y BM25.

#### **Parte 5: Comparación de Resultados**
1. Relevancia:
   - Analiza cuál de las estrategias (TF-IDF, BM25, FAISS o ChromaDB) recupera documentos más relevantes para diferentes consultas.
2. Ventajas y limitaciones:
   - Reflexiona sobre los puntos fuertes y débiles de cada enfoque.